In [14]:
import collections

start_time = 0
end_time = 10000
time_resolution = 250
respiratory_rate = 10
Pi = 15
PEEP = 5
IE = 0.5

Ventilator_log = collections.namedtuple('Patient_log', ['time', 'pressure_mouth', 'pressure_alveolus', 'pressure_intrapleural', 'lung_volume'])

class Patient:
    def __init__(self, height = 175, weight = 70, sex = 'M', pressure_mouth = 0):
        self.height = height
        self.weight = weight
        self.sex = sex
        self.lung_volume = 2000 # ml # todo - calculate static pressure
        self.pressure_mouth = pressure_mouth
        self.log = []
    
    def status(self):
        return {
            Ventilator_log(self.time, self.pressure_mouth, self.pressure_alveolus, self.pressure_intrapleural, self.lung_volume)
        }
    
    def advance(self, advance_time = 200, pressure_mouth = 0):
        self.time = self.time + advance_time # miliseconds
        
        status = self.status()
        self.log.append(status)
        return status
        
        
Ventilator_log = collections.namedtuple('Ventilator_log', ['time', 'phase', 'pressure', 'pressure_mouth'])
class Ventilator:
    def __init__(self, mode = "PCV", Pi = 15, PEEP = 5, rate = 10, IE=0.5):
        self.mode = mode
        self.Pi = Pi
        self.mode = mode
        self.PEEP = PEEP
        self.rate = rate
        self.IE = IE
        self.phase = "E"
        self.log = []
        self.time = 0 # miliseconds
    
    def target_pressure(self):
        return self.PEEP if self.phase == "E" else self.Pi
    
    def status(self):
        return {
            Ventilator_log(self.time, self.phase, self.pressure, self.pressure_mouth)
        }
    def advance(self, advance_time = 200, pressure_mouth = 0):
        self.time = self.time + advance_time # miliseconds
        self.pressure_mouth = pressure_mouth # cmH2O
        # set phase
        breath_length = 60000 / self.rate # milliseconds
        time_since_inspiration_began = self.time % breath_length
        inspiration_length = breath_length * self.IE / (self.IE + 1)
        new_phase = "I" if time_since_inspiration_began < inspiration_length else "E"
        if new_phase != self.phase:
            self.phase = new_phase
            self.pressure = self.target_pressure()
        status = self.status()
        self.log.append(status)
        return status
    
p = Patient()
v = Ventilator()

for current_time in range(start_time, end_time, time_resolution): 
    print(current_time)
    print(v.advance(advance_time = time_resolution))
    


0
{Ventilator_log(time=250, phase='I', pressure=15, pressure_mouth=0)}
250
{Ventilator_log(time=500, phase='I', pressure=15, pressure_mouth=0)}
500
{Ventilator_log(time=750, phase='I', pressure=15, pressure_mouth=0)}
750
{Ventilator_log(time=1000, phase='I', pressure=15, pressure_mouth=0)}
1000
{Ventilator_log(time=1250, phase='I', pressure=15, pressure_mouth=0)}
1250
{Ventilator_log(time=1500, phase='I', pressure=15, pressure_mouth=0)}
1500
{Ventilator_log(time=1750, phase='I', pressure=15, pressure_mouth=0)}
1750
{Ventilator_log(time=2000, phase='E', pressure=5, pressure_mouth=0)}
2000
{Ventilator_log(time=2250, phase='E', pressure=5, pressure_mouth=0)}
2250
{Ventilator_log(time=2500, phase='E', pressure=5, pressure_mouth=0)}
2500
{Ventilator_log(time=2750, phase='E', pressure=5, pressure_mouth=0)}
2750
{Ventilator_log(time=3000, phase='E', pressure=5, pressure_mouth=0)}
3000
{Ventilator_log(time=3250, phase='E', pressure=5, pressure_mouth=0)}
3250
{Ventilator_log(time=3500, phase='E